# Entrenamiento y prueba de un modelo clasificador basado en árboles de decisión

Observando el dataset que manejamos, percibimos que tiene pocos datos y, para el problema que queremos resolver, necesitamos un clasificador binario dado que el *target* es una variable discreta que toma los valores "SI" o "NO". Para ello, podemos pensar en una primera aproximación a través de árboles de decisión, cuya interpretación es muy intuitiva y tienen alta explicabilidad. 

El dilema que encontramos aquí es que este tipo de modelos tienden a sobreajustar los datos de entrenamiento. Debido a la naturaleza del problema y a los escasos datos en general de los que partimos, pese a evaluar el rendimiento del modelo mediante división en *training* y *test*, quizás los resultados no sean concluyentes. Estudiaremos si podemos recurrir a validación cruzada y si esta técnica mejora la evaluación de resultados. 

Se ha elegido un árbol de decisión por encima de modelos más complejos como *Random Forest* debido a que tenemos pocos datos, tanto de entrenamiento como de test, y es un problema de clasificación bastante sencillo, con no muchas variables. Aún así, también probaremos con modelos algo más complejos como redes neuronales no muy densas. 

Una vez se haya entrenado el modelo, se pondrá en práctica con 4 instancias de datos cuyas etiquetas desconocemos, ya que son días que, a día de hoy (25 de julio de 2025), no se han desarrollado aún. La utilidad del modelo será poder predecir la niebla que habrá durante estos días en base a las previsiones de las que disponemos.

## Paquetes

In [2]:
import pandas as pd
import numpy as np
import sys 
import os

## Importación de datos y tratamiento

Primero, descargamos los datos de los CSV's generados con las llamadas a la API de la AEMET.

In [15]:
semilla = 123
np.random.seed(semilla)
base_dir = os.path.dirname(os.path.abspath(os.getcwd()))

datos = pd.read_csv(os.path.join(base_dir, "data", "processed", "data.csv"), sep=";")
datos_sin_etiquetar = pd.read_csv(os.path.join(base_dir, "data", "processed", "data_sin_etiquetar.csv"), sep=";")

Una vez cargados, debemos convertir las variables cualitativas a *dummies* para asegurar un correcto entrenamiento y uso del modelo. Lo haremos con la función `get_dummies` de `pandas`.

Primero, extraemos las variable categóricas de los dataset:

In [16]:
categoricas = datos.select_dtypes(include= ["object", "category"]).columns
categoricas_sin_etiquetar = datos_sin_etiquetar.select_dtypes(include= ["object", "category"]).columns
categoricas

Index(['fecha', 'altonubes', 'nubosidad', 'lluvia', 'viento', 'SUBIR'], dtype='object')

No vamos a tener en cuenta las variables "fecha", dado que esta es el índice (clave primaria que identifica cada dato); ni "SUBIR", dado que es el target a predecir y no haría falta transformarlo. De hecho, la variable "fecha" podrá eliminarse por completo del dataset, dado que no influye a la hora de realizar las predicciones.

In [17]:
categoricas = categoricas.delete([0, 5])
datos.drop("fecha", axis=1, inplace=True)
categoricas

Index(['altonubes', 'nubosidad', 'lluvia', 'viento'], dtype='object')

De las "sin etiquetar", vamos a eliminar únicamente la fecha.

In [18]:
datos_sin_etiquetar.drop("fecha", axis=1, inplace=True)
categoricas_sin_etiquetar = categoricas_sin_etiquetar.delete(0)
categoricas_sin_etiquetar

Index(['altonubes', 'nubosidad', 'lluvia', 'viento'], dtype='object')

Transformamos a dummies y eliminamos las variables originales:

In [19]:
categoricas_dummies = pd.get_dummies(datos[categoricas], drop_first = True, dummy_na = True)
categoricas_sin_etiquetar_dummies = pd.get_dummies(datos_sin_etiquetar[categoricas_sin_etiquetar], drop_first = True, dummy_na = True)

datos.drop(categoricas, axis = 1, inplace = True)   # inplace para que se elimine sobre el dataset
datos_sin_etiquetar.drop(categoricas_sin_etiquetar, axis=1, inplace=True)

# Tras eliminar los atributos originales, concatenamos los nuevos atributos creados para las variables categóricas.
datos = pd.concat([datos, categoricas_dummies], axis = 1)
datos_sin_etiquetar = pd.concat([datos_sin_etiquetar, categoricas_sin_etiquetar_dummies], axis = 1)

# Mostramos cómo queda tras este proceso
datos.head()

,prec,tmin,tmax,velmedia,hrMedia,hrMax,hrMin,SUBIR,altonubes_mid,altonubes_nan,nubosidad_escasa,nubosidad_media,nubosidad_nan,lluvia_no,lluvia_posible,lluvia_nan,viento_fuerte,viento_moderado,viento_nan
0,0.0,11.9,20.4,1.4,99.0,100.0,61.0,NO,False,False,False,False,False,False,True,False,False,True,False
1,2.0,11.7,25.0,2.5,64.0,100.0,40.0,NO,False,True,False,False,True,False,False,True,False,False,True
2,23.8,13.7,26.6,5.6,54.0,94.0,34.0,SI,True,False,True,False,False,False,False,False,True,False,False
3,2.0,11.5,19.9,3.6,86.0,100.0,50.0,SI,False,True,False,False,False,False,False,False,False,True,False
4,0.0,11.4,18.2,1.4,95.0,100.0,80.0,NO,False,False,True,False,False,True,False,False,False,False,False


## Construcción del modelo

Una vez preparados los datos, vamos a proceder con el entrenamiento del modelo. Para ello, usaremos los datos etiquetados de los que disponemos, que conforman un conjunto extremadamente pequeño. Debido a esto, no conviene realizar un *train-test split* al uso, sino que optamos por una validación cruzada con *Leave-One-Out*. Esta es la mejor forma de lidiar con datasets pequeños, dado que cada instancia se usará una vez como test, y el resto como entrenamiento. Es una técnica costosa computacionalmente, pero ante la escasa cantidad de datos, no será un inconveniente.